# API 연습

## github서버에서 가져오기

대표적인 공공API

공공데이터포털: https://www.data.go.kr  
문화데이터 광장: https://www.culture.go.kr/data/main/main.do#main  
공간정보 오픈플랫폼: https://www.vworld.kr/dev/v4api.do  
금융감독원 오픈 API: https://opendart.fss.or.kr/  
네이버 오픈 API: https://developers.naver.com/products/intro/plan/plan.md  
카카오 디벨로퍼: https://developers.kakao.com/tool  
서울 열린데이터 광장: https://data.seoul.go.kr/together/guide/useGuide.do  
한국은행 Open API: https://ecos.bok.or.kr/api/#/  

In [1]:
#get매서드의 arguments로 첫번째는 URL, 두번째는 keyword방식으로 headers값을 설정해주었음
#status_code를 통해 서버와의 연결상태 확인(200은 정상,https://httpstatuses.com/200에 들어가서 200만 바꿔주면 오류 파악이 가능하다)
#parameter를 주지 않으면 요청에 대해 처리할수 없음(422)반환
#HTTP통신위해 requests 모듈 사용
import requests

response = requests.get('https://api.github.com/repositories',
                        headers={'Accept':'application/vnd.github.v3+json'})
print(response.status_code)

200


In [2]:
#response에 대한 요소 확인
print(f"인코딩: {response.encoding}")
print(f"콘텐츠 타입: {response.headers['Content-Type']}")
print(f"서버: {response.headers['server']}")

인코딩: utf-8
콘텐츠 타입: application/json; charset=utf-8
서버: GitHub.com


In [5]:
#json으로 파싱작업을 통해 일련의 문자열을 토큰으로 분해하고 이들로 이루어진 파스 트리를 만들어줌
#컨텐츠 타입이 json ==> json으로 변환 가능한 객체 ==>
# ==> json라이브러리를 활용하여 구조화(딕셔너리형태의 문자열(Seriealization문자열)을 딕셔너리화해 반환)
import json
print(json.dumps(response.json()[0], indent=2)[:200])#response.json == 파싱(딕셔너리 or 리스트 안에 딕셔너리 형태), json.dumps == print할때 쓰는듯?

{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [20]:
print(type(response.json()))

<class 'dict'>


In [ ]:
#public repositories에 대한 정보 전부(너무 많아 수행X)
response.json()

In [15]:
#검색API를 사용해보자
#parameter를 주지 않으면 요청에 대해 처리할수 없음(422)반환 (통신에 대한 query parameters는 API마다 다름)
response = requests.get('https://api.github.com/search/repositories')
print(response.status_code)

422


In [16]:
#param을 통해 (언어는 python)으로 data_science관련을 검색 + headers ==> 당연히 검색API니깐 검색할것을 알려줘야지
#양식을 통해 API가 url안의 쿼리문을 만들어줌
response = requests.get('https://api.github.com/search/repositories',
                       params={'q':'data_science+language:python'},
                       headers={'Accept':'application/vnd.github.v3.text-match+json'})
print(response.status_code)

200


In [21]:
response.json().keys()

dict_keys(['total_count', 'incomplete_results', 'items'])

In [22]:
#response의 key중에 items(repository에 대한 정보)의 첫번째에서 text_matches라는 키값의 value 호출
response.json()['items'][0]['text_matches']

[{'object_url': 'https://api.github.com/repositories/26382146',
  'object_type': 'Repository',
  'property': 'description',
  'fragment': 'code for Data Science From Scratch book',
  'matches': [{'text': 'Data Science', 'indices': [9, 21]}]},
 {'object_url': 'https://api.github.com/repositories/26382146',
  'object_type': 'Repository',
  'property': 'name',
  'fragment': 'data-science-from-scratch',
  'matches': [{'text': 'data', 'indices': [0, 4]}]}]

In [23]:
#items는 30개의 리스트를 value로 가짐
len(response.json()['items'])

30

## pytorch오픈소스 가져오기

pagination = 서버의 과부하를 방지하기 위해 서버에서 응답 개수를 제한  
next 30개만 보여줬음

In [3]:
response = requests.get('https://api.github.com/repos/pytorch/pytorch/issues',
                        headers={'Accept':'application/vnd.github.v3.text-match+json'})
print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 200
Number of comments 30


In [27]:
response.links

{}

오픈소스 가져올때 제한이 걸리므로 함수를 사용해서 자동으로 갱신되게 만들자

In [ ]:
#재귀제한걸기
import sys
sys.setrecursionlimit(10**5) # 10의 5승

In [26]:
#next가 없을때까지 재귀를 통해 모든 페이지를 출력(역순으로 리턴됨)
def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url,params=params, headers=headers)
    if response.status_code ==200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next+url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json

In [ ]:
#위 함수를 통해 가져온 데이터를 DataFrame화해서 결과확인
import pandas as pd

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)

In [ ]:
#제한횟수, 제한시간까지 얼마나 남았는지 출력해보자
response = requests.head('https://api.github.com/repos/pytorch/pytorch/issues/comments')
print('X-Ratelimit-Limit', response.headers['X-Ratelimit-Limit'])
print('X-Ratelimit-Remaining', response.headers['X-Ratelimit-Remaining'])

# UTC 시간을 사람이 읽을 수 있는 형식으로 변환
import datetime
print('Rate Limit reset at', datetime.datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset'])).strftime('%c'))

API 호출 속도 조절하기  

time 라이브러리의 sleep() 메서드를 활용하여 특정 시간동안 대기하도록 할 수 있습니다.  
앞서 이야기한 X-RateLimit-Reset과 X-Ratelimit-Remaining의 값을 활용하여 대기시간을 구합니다.  
아래의 코드는 그 대기시간동안 기다렸다가 API 호출을 하는 handle_rate_limits 함수를 구현하였습니다.  

In [ ]:
from datetime import datetime
import time

def handle_rate_limits(response):
    now = datetime.now()
    reset_time = datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time = (reset_time - now).total_seconds()
    intervals = remaining_time / (1.0 + int(remaining_requests))

    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True

네트워크 오류를 감안한 코드 작성1  

API 호출에는 여러가지 변수가 존재합니다. 연결 중단, DNS 조회 실패, 연결 시간 초과 등이 있습니다.  
호출 실패에 대한 재시도를 할 수 있도록 HTTPAdapter를 사용하는데, Retry 객체를 통해 초기화합니다.  
total은 재시도 횟수, status_forcelist는 재시도할 상태 코드 목록을, backoff_factor는 각 시도마다 간격을 늘려주는 수치입니다.

In [ ]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
    total=5,
    status_forcelist=[500, 503, 504],
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)
#연결 동작에 재시도를 추가하려면 재시도를 사용할 수 있는 사용자 지정 어댑터를 지정해야 합니다.
#그럴 때는 우측의 코드와 같이 새로운 http Session 객체를 생성해서 사용하면 됩니다.
http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("https://", retry_adapter)

response = http.get('https://api.github.com/search/repositories',
                    params={'q': 'data_science+language:python'})

for item in response.json()['items'][:5]:
    print(item['name'])

In [ ]:
#완성본인듯??
#status가 500,503,504상태이면 알아서 재시작
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
    total=5,
    status_forcelist=[500, 503, 504], 
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("http://", retry_adapter)

def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = http.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if (next_url is not None) and (handle_rate_limits(response)):
                output_json += get_all_pages(next_url, params, headers)
    return output_json

# API 실습해보기

In [ ]:
#인증키 : 463cba3e96c43560eda520cfdad14d0c245d6575

In [16]:
pip install xmltodict


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import requests
from io import BytesIO
import zipfile
import xmltodict
api = 'https://opendart.fss.or.kr/api/corpCode.xml'
res = requests.get(api, params={'crtfc_key': '463cba3e96c43560eda520cfdad14d0c245d6575'})
data_xml = zipfile.ZipFile(BytesIO(res.content))

In [10]:
data_xml.namelist()

['CORPCODE.xml']

In [12]:
import json
data_xml_z = data_xml.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml_z)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

In [19]:
data[:5]

[{'corp_code': '00434003',
  'corp_name': '다코',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00434456',
  'corp_name': '일산약품',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00430964',
  'corp_name': '굿앤엘에스',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00432403',
  'corp_name': '한라판지',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00388953',
  'corp_name': '크레디피아제이십오차유동화전문회사',
  'stock_code': None,
  'modify_date': '20170630'}]

In [21]:
#corp_code 획득 00260930
for item in data:
    if item['corp_name'] == '에스엠':
        print(item)

{'corp_code': '00521178', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20170630'}
{'corp_code': '00398118', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20170630'}
{'corp_code': '01078628', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20170630'}
{'corp_code': '01101643', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20170630'}
{'corp_code': '01491917', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20200806'}
{'corp_code': '01684605', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20220913'}
{'corp_code': '00658649', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20230221'}
{'corp_code': '00260930', 'corp_name': '에스엠', 'stock_code': '041510', 'modify_date': '20230220'}
{'corp_code': '01012349', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20230202'}
{'corp_code': '01477476', 'corp_name': '에스엠', 'stock_code': None, 'modify_date': '20230203'}


In [77]:
import requests

response = requests.get('https://opendart.fss.or.kr/api/majorstock.json',
                        params={'crtfc_key': '463cba3e96c43560eda520cfdad14d0c245d6575',
                               'corp_code' : '00260930'})
print(response.status_code)

200


In [55]:
response.json()['list'][0]

{'rcept_no': '20210408000094',
 'rcept_dt': '2021-04-08',
 'corp_code': '00260930',
 'corp_name': '에스엠',
 'report_tp': '약식',
 'repror': '한국투자신탁운용',
 'stkqy': '1,182,839',
 'stkqy_irds': '-311,595',
 'stkrt': '5.04',
 'stkrt_irds': '-1.33',
 'ctr_stkqy': '-',
 'ctr_stkrt': '-',
 'report_resn': '1%이상 변동'}

In [ ]:
#'repror' = '이수만'
lsm=[]
for i in response.json()['list']:
    for key, value in i.items():
        if value =='이수만' or '하이브':
            lsm.append(i)
        
for _ in lsm:
    print('사명 :',_.get('corp_name'),'매매자 :',_.get('repror'),'보유비율 증감 :',_.get('stkrt_irds'))

# 웹크롤링

## robots.txt 분석하기

In [1]:
#로이터통신 주소의 robots.txt가져옴
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.reuters.com/robots.txt")
rp.read()

In [5]:
# 특정 User-agent가 url에 접근 가능한지 확인한다.
rp.can_fetch(useragent="*",url="https://reuters.com/sitemap.xml")

True

In [6]:
#sitemap에 대한 정보 확인 ==>xml형태임을 확인
sitemaps = rp.site_maps()
sitemaps

['https://www.reuters.com/arc/outboundfeeds/sitemap-index/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap-index/?outputType=xml',
 'https://www.reuters.com/sitemap_video_index.xml',
 'https://www.reuters.com/brandfeature/sitemap']

In [1]:
!pip install xmltodict

In [8]:
import xmltodict
import requests

url = sitemaps[0] # /sitemap/lastest-articles
sitemaps = xmltodict.parse(requests.get(url).text)# xml을 dict형태로

In [39]:
#여러개의 신문기사 주소를 찾음
sitemap_urls=[]
for _ in sitemaps['sitemapindex']['sitemap']:
    sitemap_urls.append(_['loc'])

In [41]:
url = sitemap_urls[0]
news_dict = xmltodict.parse(requests.get(url).text)


In [ ]:
news_dict

In [51]:
news_urls = [_['loc'] for _ in news_dict['urlset']['url']]
print(news_urls)

['https://www.reuters.com/markets/europe/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28/', 'https://www.reuters.com/business/finance/oaktree-capital-seeks-raise-10-billion-new-fund-ft-2023-02-28/', 'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/', 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/', 'https://www.reuters.com/lifestyle/sports/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28/', 'https://www.reuters.com/markets/asia/indias-adani-plans-repay-up-790-mln-share-backed-loans-by-march-sources-2023-02-28/', 'https://www.reuters.com/lifestyle/sports/smith-says-australia-batsmen-will-ditch-risky-tempo-third-test-2023-02-28/', 'https://www.reuters.com/technology/cryptoverse-bitcoin-moves-towards-satoshis-payment-dream-2023-02-28/', 'https://www.reuters.com/markets/europe/ecb-has-started-win-inflation-fight-lane-says-2023-02-28/', 'https://www.reuters.com/markets/rates-bonds

In [64]:
%%time
#datas 폴더에 new_urls의 html파일들을 가져옴(스크래핑)
#session 추상화
session = requests.Session()

#앞의 5개의 링크만 가져와봅시다
for url in news_urls[:5]:
    file = url.split("/")[-2] + '.html'
    
    response = session.get(url)
    if response.ok:
        with open("datas/"+file, "w+b") as f:
            f.write(response.text.encode('utf-8'))
    else:
        print(f"error widh URL : {URL}")

CPU times: total: 125 ms
Wall time: 1.96 s


## 파일 사용하기

In [2]:
#filename.txt 파일을 쓰기 모드로(w)로 열기. open() 함수는 파일 객체 반환
file = open('filename.txt','w')

#파일 객체의 write() 매서드를 통해 문자열을 파일에 쓸 수 있습니다
file.write('파일에 작성할 문자열')

#파일에 대한 처리가 끝났으면 파일을 닫아줘야 합니다
file.close()

In [3]:
#close에 대한 처리 없이 간편히 쓰려면 with를 활용할 수 있습니다.
#위의 코드와 동일한 동작을 수행합니다.
with open('filename.txt', 'w') as file:
    file.write('파일에 작성할 문자열')

In [4]:
#파일 읽어오기
with open('filename.txt', 'r') as file:
    print(file.read())

파일에 작성할 문자열


In [5]:
import os
path = './datas/'
files = [path + file for file in os.listdir(path)]
files

['./datas/.ipynb_checkpoints',
 './datas/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28.html',
 './datas/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28.html',
 './datas/global-markets-view-europe-2023-02-28.html',
 './datas/home-zara-fast-slow-fashion-collide-2023-02-28.html',
 './datas/oaktree-capital-seeks-raise-10-billion-new-fund-ft-2023-02-28.html']

In [6]:
with open(files[1], "r", encoding='utf-8') as f:#두번째파일을 불러와서
    html = f.read() #html변수에 담음

## 기사 헤드라인 추출하기

In [7]:
#BeaurifulSoup == HTML을 파싱하는데 필요한 모듈 첫번째 인수로 파싱할 대상인 mark up 을 두번째 인수로 무엇을 기준으로 파싱할지 파서의 종류("html.parser","lxml","xml"등등)를 적습니다
#웹상에서 가져올 기사의 기사제목을 f12 -> ctrl+shift+c로 개발자모드로 가서 copy-> copy selector로 가져왔음
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select_one("#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > header > div > div > h1")
print(title.text)

Bayer sees lower 2023 operating profit on cost inflation


In [8]:
#기사제목은 ==> <h1>텍스트</h1>형태로 주어짐

In [9]:
soup.h1 # soup.h1을 조회하면 <h1> 태그에 해당하는 객체를 가져올 수 있으며,

<h1 class="text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_3__1kDhc heading__base__2T28j heading__heading_3__3aL54 article-header__title__3Y2hh" data-testid="Heading">Bayer sees lower 2023 operating profit on cost inflation</h1>

In [10]:
soup.title.text #sou.title.text라고 하면 <h1> 태그 사이에 있는 텍스트 값을 가져올 수 있습니다

'Bayer sees lower 2023 operating profit on cost inflation | Reuters'

In [ ]:
#본문기사중 부분내용 선택해서 가져오기  
#<p>text</p>형태로 가지고있음

In [14]:
#selector copy --> soup.select_one을 이용해 가져왔음
soup.select_one("#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > div > div > div.article-body__content__17Yit.paywall-article > p:nth-child(2)").text

"FRANKFURT, Feb 28 (Reuters) - Agriculture and healthcare company Bayer (BAYGn.DE) said operating earnings would likely decline in 2023, hurt by higher costs and the reversal of last year's price boost for its glyphosate-based weedkillers."

## URL 추출
많은 HTML 파일을 다운로드할 때 파일의 원본 URL을 별도로 저장하지 않으면 나중에 찾기 어렵습니다.  
일반적으로 표준 URL(canonical URL)을 사용해서 값을 관리하는 것이 좋습니다  
이 용도에 맞는 태그가 별도로 존재하는데 '<'link rel="canonical"'>'이라고 사용됩니다

In [19]:
soup.find("link", {"rel": "canonical"})["href"]

'https://www.reuters.com/business/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28/'